In [30]:
%pip install gradio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\natha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [31]:
import whisper
import gradio as gr

C:\Users\natha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
def transcribe(audio_file):
    model = whisper.load_model('base')
    result = model.transcribe(audio_file)
    return result['text']

# LangChain

In [12]:
%pip install chromadb==0.4.15
%pip install langchainhub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\natha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\natha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import getpass
openai_api_key = getpass.getpass('Enter OpenAI API key:')

In [5]:
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [6]:
loader = CSVLoader(file_path='chunk_text_db.csv', csv_args={
    'delimiter': ',',
    'quotechar': '"',
})
data = loader.load()

In [7]:
print(data)

[Document(page_content='transcript: transcript\nlocation: location', metadata={'source': 'chunk_text_db.csv', 'row': 0}), Document(page_content='transcript: the Gospel according to Mark. Chapter 1\nlocation: mark1-0.mp3', metadata={'source': 'chunk_text_db.csv', 'row': 1}), Document(page_content="transcript: the beginning of the gospel of Jesus Christ, the Son of God. As it is written in Isaiah the prophet, behold I send my messenger before your face, who will prepare your way, the voice of one crying in the wilderness, prepare the way of the Lord, make His paths straight. John appeared, baptizing in the wilderness and proclaiming a baptism of repentance for the forgiveness of sins, and all the country of Judea and all Jerusalem were going out to Him and were being baptized by Him in the river Jordan, confessing their sins. Now John was clothed with camel's hair and wore a leather belt around his waist and ate locusts and wild honey, and he preached, saying, after me comes He who is mi

In [8]:
data[1].page_content

'transcript: the Gospel according to Mark. Chapter 1\nlocation: mark1-0.mp3'

In [9]:
vectorstore = Chroma.from_documents(documents=data, 
                                    embedding=OpenAIEmbeddings(openai_api_key=openai_api_key))

In [10]:
retriever = vectorstore.as_retriever()

## Retrieve Relevant Audio

In [42]:
def retrieve_audio_location(doc):
    split_doc = doc.rsplit(' ', 1)
    return 'C:/Users/natha/OneDrive/Desktop/coding projects/audio_search/audio_chunks/' + split_doc[1]

In [43]:
def retrieve_audio_chunk(query):
    search = transcribe(query)
    retrieved_docs = retriever.get_relevant_documents(
    search
    #query
    )
    best_fit = retrieved_docs[0].page_content
    audio_path = retrieve_audio_location(doc=best_fit)
    return audio_path

In [44]:
retrieve_audio_chunk('who was the brother of andrew?')

C:\Users\natha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


RuntimeError: Failed to load audio: ffmpeg version 6.1-essentials_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --pkg-config=pkgconf --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-dxva2 --enable-d3d11va --enable-libvpl --enable-libgme --enable-libopenmpt --enable-libopencore-a

In [45]:
gr.Interface(fn=retrieve_audio_chunk, 
            inputs=gr.Audio(sources=['microphone', 'upload'], type='filepath'), 
            outputs='audio').launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\natha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Keyboard interruption in main thread... closing server.


## QA

In [19]:
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)

In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [16]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
rag_chain.invoke("Who was the brother of Andrew?")

'The brother of Andrew was Simon.'

TTS:

In [47]:
%pip install --upgrade transformers sentencepiece datasets[audio]

     ---------------------------------------- 2.1/2.1 MB 7.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\natha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     -------------------------------------- 123.5/123.5 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 9.1 MB/s eta 0:00:00
     ---------------------------------------- 61.2/61.2 kB ? eta 0:00:00
   ---------------------------------------- 7.9/7.9 MB 26.6 MB/s eta 0:00:00
   --------------------------------------- 311.7/311.7 kB 18.8 MB/s eta 0:00:00
   --------------------------------------- 521.2/521.2 kB 34.1 MB/s eta 0:00:00
   --------------------------------------- 253.7/253.7 kB 15.2 MB/s eta 0:00:00
   ---------------------------------------- 15.8/15.8 MB 40.9 MB/s eta 0:00:00
   --------------------------------------- 181.6/181.6 kB 11.4 MB/s eta 0:00:00
   ---------------------------------------- 222.8/222.8 kB ? eta 0:00:00
   ---------------------------------------- 62.7/62.7 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 184.6/184.6 kB ? eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\natha\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\transformers\\models\\deprecated\\trajectory_transformer\\convert_trajectory_transformer_original_pytorch_checkpoint_to_pytorch.py'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [48]:
from transformers import pipeline
from datasets import load_dataset
import soundfile as sf

ImportError: cannot import name 'DEFAULT_ETAG_TIMEOUT' from 'huggingface_hub.constants' (C:\Users\natha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\constants.py)

In [ ]:
synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts")

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
# You can replace this embedding with your own as well.

speech = synthesiser("Hello, my dog is cooler than you!", forward_params={"speaker_embeddings": speaker_embedding})

sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])

# Gradio

In [ ]:
gr.Interface(fn=transcribe, 
            inputs=gr.Audio(sources=['microphone', 'upload'], type='filepath'), 
            outputs=[
                'text'
                ]).launch(debug=True)